In [2]:
from astropy.io import fits
import os
import subprocess
import numpy as np

In [18]:
#imageDirectory = '/data/mj1e16/kepler/cal/'
# diffDir = '/data/mj1e16/kepler/properDiff/'
# medianDir = '/data/mj1e16/kepler/medians/'
# imageDir = '/data/mj1e16/kepler/cal/'

diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
medianDir = '/media/mj1e16/PP AV-TV/medians/'
imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'
imageDirectory = '/media/mj1e16/PP AV-TV/keplerCal/'

In [ ]:
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0]!='.']
dirlist = [imageDirectory+x for x in dirlist]

In [9]:
supernovaStart = [56176.666,55846.320,55928.414]

In [10]:
def findClosestImage(supernovaStart,dirlist):
    answer = []
    for y in range(len(supernovaStart)):
        timediff = []
        for x in range(len(dirlist)):
            #print(dirlist[x])
            hdu = fits.open(dirlist[x])
            header = hdu[1].header
            timediff.append(abs(header['MJDSTART']-supernovaStart[y]))

        closest = min(timediff)
        imageIndex =  timediff.index(closest)   
        answer.append({'timediff':closest,'index':imageIndex,'image':dirlist[imageIndex]})
    return answer

In [11]:
answer = findClosestImage(supernovaStart,dirlist)

In [12]:
print(answer)

[{'index': 45, 'timediff': 7.854872060001071, 'image': '/media/mj1e16/PP AV-TV/keplerCal/kplr2012242195726_ffi-cal.fits'}, {'index': 34, 'timediff': 13.539371960003336, 'image': '/media/mj1e16/PP AV-TV/keplerCal/kplr2011271191331_ffi-cal.fits'}, {'index': 37, 'timediff': 2.427519040000334, 'image': '/media/mj1e16/PP AV-TV/keplerCal/kplr2012004204112_ffi-cal.fits'}]


In [13]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [14]:
extensionOrders = []
for image in dirlist:
    extensionOrders.append(correctExtensionOrder(image))

In [15]:
def makeDS9RegFile(xcoords,ycoords,fileNameBase):
    
    for x in range(len(xcoords)):
        bigString = 'global color=lightgreen\nWCS\n'
        bigString += 'circle({},{},0.0015)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase+str(x)+'.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [ ]:
def findName(image,diffDir=diffDir,medianDir=medianDir):
    hdu = fits.open(diffDir+image)
    head = hdu[0].header['DIFFCMD']
    locator1 = '/data/mj1e16/kepler/cal/'
    locator2 = ']'
    loc1 = head.find(locator1) + len(locator1)
    loc2 = head.find(locator2) + 1
    imname = head[loc1:loc2]
    
    channellocator1 = 'channel' 
    channellocator2 = '_ccd'
    ccdlocator1 = 'ccd'
    ccdlocator2 = '.fits'
    loc1 = image.find(channellocator1) + len(channellocator1)
    loc2 = image.find(channellocator2)
    channel = image[loc1:loc2]
    loc1 = image.find(ccdlocator1) + len(ccdlocator1)
    loc2 = image.find(ccdlocator2)
    ccd = image[loc1:loc2]
    
    medianName = 'meidian_{}_{}.fits'.format(channel,ccd)
    results = [imname,medianName]
    return results

In [16]:
names = ['KSN 2012a','KSN 2011b','KSN 2011c'] 
# ccds = [61,42,29]
# ccds = [37,41,81]
#ccds = [21,44,63]
ccds = [45,1,44]
ra = [293.3754458333333, 290.15641666666664, 291.1920374999999]
dec = [45.25035833333333, 38.252316666666665, 43.68074444444444]

In [17]:
makeDS9RegFile(ra,dec,'supernovae')

In [43]:
answer[0]['image']+str(ccds[0])

'/data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits61'

In [76]:
for x in range(len(names)):
    #extension = int(extensionOrders[answer[x]['index']][ccds[x]])
    extension = ccds[x]
    print(extension)
    print('ds9 '+answer[x]['image']+'[{}]'.format(extension)+' -regions supernovae{}.reg'.format(x))
    #subprocess.call(['ds9',answer[x]['image']+'[{}]'.format(ccds[x])]) #,'-regions','supernovae{}.reg'.format(x)]

45
ds9 /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -regions supernovae0.reg
1
ds9 /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -regions supernovae1.reg
43
ds9 /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[43] -regions supernovae2.reg


In [ ]:
# ds9 /data/mj1e16/kepler/cal/kplr2012242195726_ffi-cal.fits[45] -regions supernovae0.reg
# ds9 /data/mj1e16/kepler/cal/kplr2011271191331_ffi-cal.fits[1] -regions supernovae1.reg
# ds9 /data/mj1e16/kepler/cal/kplr2012004204112_ffi-cal.fits[44] -regions supernovae2.reg

In [80]:
def hotpants(imNo,channel,extensionOrders,imagelist,name,medianDir='/data/mj1e16/kepler/medians/'):
    ccdChannel = int(extensionOrders[imNo][channel])
    inim = imagelist[imNo]+'[{}]'.format(ccdChannel)
    tmplim = medianDir+'meidian_{}.fits'.format(channel)
    print(tmplim)
#     outim = '/home/mj1e16/testImages/supernovaDiff_'+name+'.fits'
#     # outim = '/data/mj1e16/kepler/properDiff/diff_imNo{}_channel{}.fits'.format(imNo,channel)
#     subprocess.call(['./hotpants','-inim',inim,'-tmplim',tmplim,'-outim',outim,'-v','0'])